In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install --upgrade scipy networkx numpy

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 48.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
#from torch_geometric.utils.train_test_split_edges import torch_geometric
#torch_geometric.transforms.RandomLinkSplit
import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from scipy.sparse import csr_matrix
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import matrix_util as Mat_Util
import DGCNN_layer as dgcnn_layer
import DGCNN_GNN_Layers as dgcnn_gnn_layers
import SortPooling_Layer as sortpooling_layer
import MLP_DGCNN as mlp_dgcnn

class GlobalMeanPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return gnn.global_mean_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class DGCNN_Model(nn.Module):
    '''
        DGCNN Layers using sparse adjacency matrix
    '''
    def __init__(self, GNN_layers, mlp_act_fun, dgcnn_act_fun, mlp_dropout_rate, Weight_Initializer, Bias, num_classes, dgcnn_k,
                 node_feat_size, hid_channels, conv1d_kernels, ffn_layer_size, strides):

        super(DGCNN_Model, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.GNN_layers = GNN_layers
        self.output_dim = GNN_layers[-1]
        self.num_GNN_layers = len(GNN_layers)
        self.mlp_dropout_rate = mlp_dropout_rate
        self.Bias = Bias
        self.Weight_Initializer = Weight_Initializer
        self.dgcnn_k = dgcnn_k
        self.node_feat_size = node_feat_size
        self.num_classes = num_classes
        self.hid_channels = hid_channels
        self.conv1d_kernels = conv1d_kernels
        self.ffn_layer_size = ffn_layer_size
        self.strides = strides

        self.gnn_layers = dgcnn_gnn_layers.dgcnn_gnn_layers(GNN_layers=self.GNN_layers, node_feat_size=self.node_feat_size,
                                                            Bias=self.Bias, dgcnn_act_fun=dgcnn_act_fun).to(self.device)

        self.sort_pool = sortpooling_layer.SortPooling(self.dgcnn_k, node_feat_size=self.node_feat_size).to(self.device)

        self.classic_conv = mlp_dgcnn.MLP_DGCNN(num_class=self.num_classes, last_gnn_layer_dim=self.GNN_layers[-1],
                                                mlp_act_fun=mlp_act_fun, dropout_rate=self.mlp_dropout_rate,
                                                hid_channels=self.hid_channels, conv1d_kernels=self.conv1d_kernels,
                                                dgcnn_k=self.dgcnn_k, ffn_layer_size=self.ffn_layer_size, Bias=self.Bias,
                                                strides=self.strides).to(self.device)
        if dgcnn_act_fun == 'ReLu':
            self.dgcnn_act_fun = F.relu
            print('ReLu is Selected.')
        elif dgcnn_act_fun == 'eLu':
            self.dgcnn_act_fun = nn.functional.elu
            print('eLu is Selected.')
        elif dgcnn_act_fun == 'tanh':
            self.dgcnn_act_fun = torch.tanh
            print('tanh is Selected.')



        mean = 0
        std = 0.1
        #self.initialize_weights(Weight_Initializer, Bias, mean, std)


    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.xavier_normal_(layer.weight.data)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.xavier_normal_(layer.conv_params.weight)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.xavier_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.xavier_normal_(layers.conv1d_1.weight)
                    torch.nn.init.xavier_normal_(layers.conv1d_2.weight)
                    torch.nn.init.xavier_normal_(layers.linear1.weight)
                    torch.nn.init.xavier_normal_(layers.linear2.weight)

                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.kaiming_normal_(layer.weight.data)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.kaiming_normal_(layer.conv_params.weight)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.kaiming_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.kaiming_normal_(layers.conv1d_1.weight)
                    torch.nn.init.kaiming_normal_(layers.conv1d_2.weight)
                    torch.nn.init.kaiming_normal_(layers.linear1.weight)
                    torch.nn.init.kaiming_normal_(layers.linear2.weight)

                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        #print("here2")
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.normal_(layer.weight.data, mean, std)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.normal_(layer.conv_params.weight, mean, std)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.normal_(layers.weight, mean, std)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.normal_(layers.conv1d_1.weight, mean, std)
                    torch.nn.init.normal_(layers.conv1d_2.weight, mean, std)
                    torch.nn.init.normal_(layers.linear1.weight, mean, std)
                    torch.nn.init.normal_(layers.linear2.weight, mean, std)
                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass


    def forward(self, graph, edge_mask):

        if graph.batch is not None:
            graph_sizes = [len(graph[i].x) for i in range(len(graph))]
        else:
            graph_sizes = [len(graph.x)]


        Output_of_GNN_Layers = self.gnn_layers(graph, edge_mask).to(self.device)



        Output_of_GNN_Layers.retain_grad()


        sortpooled_embedings = self.sort_pool(output_of_dgcnn_layer=Output_of_GNN_Layers, batch_graphs=graph)

        sortpooled_embedings.retain_grad()


        output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2 = self.classic_conv(sortpooled_embedings=sortpooled_embedings, graph_sizes=graph_sizes)

        return Output_of_GNN_Layers, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2






#node_feat_size = len(dataset[0].x[0])
#k=17
#dgcnn_model_example = DGCNN_Model(GNN_layers=[32, 32, 32, 7], num_classes=2, mlp_act_fun='ReLu', dgcnn_act_fun='tanh', mlp_dropout_rate=0.5, Weight_Initializer=3, Bias=False, dgcnn_k=k, node_feat_size=node_feat_size, hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])

#print(dgcnn_model_example)
#final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = dgcnn_model_example(dataset[0], None)
#print(softmaxed_h2)

dataset = TUDataset(root='data/TUDataset', name='MUTAG')
batch_size = 2
k = 17
node_feat_size = len(dataset[0].x[0])
batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)
dgcnn_model_example = DGCNN_Model(GNN_layers=[32, 32, 32, 7], num_classes=2, mlp_act_fun='ReLu', dgcnn_act_fun='tanh', mlp_dropout_rate=0.5, Weight_Initializer=3, Bias=False, dgcnn_k=k, node_feat_size=node_feat_size, hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])
import random
em = [random.uniform(0, 1) for i in range(len(dataset[0].edge_index[0]))]

for batched_graphs in dataset:
    em = [random.uniform(0, 1) for i in range(len(batched_graphs.edge_index[0]))]
    #x, edge_index, batch, y = batched_graphs.x, batched_graphs.edge_index, batched_graphs.batch, batched_graphs.y
    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = dgcnn_model_example(batched_graphs, None)
    print("lin2_output_softmaxed: ", softmaxed_h2.size())
    print(softmaxed_h2)
    break

#node_feat_size = len(dataset[0].x[0])
#k=20
#dgcnn_model_example = DGCNN_Model(GNN_layers=[32, 32, 32, 7], num_classes=2, mlp_act_fun='ReLu', dgcnn_act_fun='tanh', mlp_dropout_rate=0.5, Weight_Initializer=3, Bias=False, dgcnn_k=k, node_feat_size=node_feat_size, hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])

#print(dgcnn_model_example)
#final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = dgcnn_model_example(dataset[0])
#print(softmaxed_h2)

Processing...
Done!


tanh is Selected.
tanh is Selected.
lin2_output_softmaxed:  torch.Size([1, 2])
tensor([[0.4994, 0.5006]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [ ]:
graph_example = dataset[0]
em = [random.uniform(0, 1) for i in range(len(graph_example.edge_index[0]))]
x = to_scipy_sparse_matrix(graph_example.edge_index).multiply(1)
row_A = np.array(graph_example.edge_index[0])
col_A = np.array(graph_example.edge_index[1])
data_A = np.array(em)

csrMatrix_A = csr_matrix((data_A, (row_A, col_A)))
print(csrMatrix_A)



a = csr_matrix((np.array(em), (np.array(graph_example.edge_index[0]), np.array(graph_example.edge_index[1]))))

  (0, 1)	0.035092147772491145
  (0, 5)	0.7143755231591802
  (1, 0)	0.7163840463028429
  (1, 2)	0.4052103474807921
  (2, 1)	0.4759074134777955
  (2, 3)	0.9941871848865929
  (3, 2)	0.4848262557143126
  (3, 4)	0.23738776448995058
  (3, 9)	0.9362373626887639
  (4, 3)	0.32927258797738346
  (4, 5)	0.22334926882361894
  (4, 6)	0.7925484914913264
  (5, 0)	0.8428135088426392
  (5, 4)	0.5043848497923586
  (6, 4)	0.7822545594070558
  (6, 7)	0.3032604136659791
  (7, 6)	0.023906806331804975
  (7, 8)	0.5925565381607069
  (8, 7)	0.1494193696025985
  (8, 9)	0.7413661178556381
  (8, 13)	0.9390891329608302
  (9, 3)	0.17248293721094798
  (9, 8)	0.006033802543148892
  (9, 10)	0.19392451338635108
  (10, 9)	0.06388803589475378
  (10, 11)	0.16226032350304198
  (11, 10)	0.4182358332682746
  (11, 12)	0.31465224942320935
  (12, 11)	0.8111245893280435
  (12, 13)	0.7622638644632717
  (12, 14)	0.9620399981035909
  (13, 8)	0.4861888369259414
  (13, 12)	0.910748003872269
  (14, 12)	0.28029369087168854
  (14, 15)	0.6

In [ ]:
def get_hyperparameter_K_for_DGCNN(dataset):
    num_nodes_list = []
    for graph in dataset:
        num_nodes_list.append(len(graph.x))
    #print(num_nodes_list)
    #print(max(num_nodes_list))
    my_threshold = [[],[]]
    for threshold_size in range(max(num_nodes_list)):
        #x = [i for i in num_nodes_list if i > size]
        candidates = []
        for s in num_nodes_list:
            if s >= threshold_size:
                candidates.append(s)
                if round(len(candidates)/len(num_nodes_list), 0) >= 0.6:
                    my_threshold[0].append(threshold_size) # threshold values
                    my_threshold[1].append(round(len(candidates)/len(num_nodes_list), 0) >= 0.6) # fraction rate
                    #print("threshold_size: ", threshold_size, )
                    break
                else:
                    pass

        #print(round(len(candidates)/len(num_nodes_list)*100, 0), "%", "have more node count than ", threshold_size)
    return my_threshold[0][-1]

K_DGCNN_HyperParameter = get_hyperparameter_K_for_DGCNN(dataset)
print(K_DGCNN_HyperParameter)

17
